In [28]:
# WandB – Install the W&B library
!pip install wandb -q
!pip install opencv-python
!pip install imageio
!pip install onnxruntime
!pip install keras2onnx



  Using cached onnxruntime-1.11.0-cp38-cp38-win_amd64.whl (5.5 MB)
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115928 sha256=32dc38428473a74b353e00b95e51b19a599d8ea5a4395ffdd77cb3d6ab6381f4
  Stored in directory: c:\users\avi\appdata\local\pip\cache\wheels\1f\10\06\2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
Successfully built fire


In [30]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical

import wandb
from wandb.keras import WandbCallback

import tf2onnx
import onnx

import imageio, glob, cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [10]:
(train_images, train_labels),(test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

In [11]:
# Reshape the images.
train_images_re = np.expand_dims(train_images, axis=3)
test_images_re = np.expand_dims(test_images, axis=3)

classes = np.unique(train_labels)
input_shape = train_images_re[0].shape

print(classes)
print(input_shape)


[0 1 2 3 4 5 6 7 8 9]
(28, 28, 1)


In [4]:
plt.figure(figsize=(10,10))
plt.subplots_adjust(wspace=0, hspace=0)
i = 0
for item in train_images[:25]:
    print(item[0].shape)
    w, h = 28, 28
    data = np.zeros((h, w, 3), dtype=np.uint8)
    data[0:28, 0:28] = [item[0], item[1], 0]
    #img = cv2.imread(item)
    #img = cv2.resize(img, (250, 250))
    #plt.axis('off')
    #plt.subplot(5, 5, i+1) #.set_title(l)
    #plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    i += 1

(28,)


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

<Figure size 720x720 with 0 Axes>

In [12]:
def createModel(input_shape, num_classes):
    model = keras.Sequential([keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dense(num_classes)])
    return model

In [13]:
model = createModel(input_shape, len(classes))

In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               

In [16]:
num_epochs=10
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images_re, train_labels, epochs=num_epochs, validation_split=0.2, verbose = 1)

Epoch 1/10
1500/1500 [==============================] - 53s 35ms/step - loss: 0.6914 - accuracy: 0.7990 - val_loss: 0.4553 - val_accuracy: 0.8403
Epoch 2/10
1500/1500 [==============================] - 53s 35ms/step - loss: 0.3696 - accuracy: 0.8662 - val_loss: 0.3512 - val_accuracy: 0.8685
Epoch 3/10
1500/1500 [==============================] - 44s 30ms/step - loss: 0.3252 - accuracy: 0.8819 - val_loss: 0.3369 - val_accuracy: 0.8792
Epoch 4/10
1500/1500 [==============================] - 49s 32ms/step - loss: 0.3001 - accuracy: 0.8897 - val_loss: 0.3433 - val_accuracy: 0.8733
Epoch 5/10
1500/1500 [==============================] - 49s 32ms/step - loss: 0.2824 - accuracy: 0.8956 - val_loss: 0.3162 - val_accuracy: 0.8875.2824 - accuracy: 0.89
Epoch 6/10
1500/1500 [==============================] - 45s 30ms/step - loss: 0.2610 - accuracy: 0.9028 - val_loss: 0.3071 - val_accuracy: 0.8873
Epoch 7/10
1500/1500 [==============================] - 44s 29ms/step - loss: 0.2526 - accuracy: 0.905

In [20]:
test_loss, test_acc = model.evaluate(test_images_re,  test_labels, verbose=2)

313/313 - 2s - loss: 0.3246 - accuracy: 0.8904


In [35]:
input_sig = [tf.TensorSpec([3,3], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_sig, opset=13)
#onnx.save(onnx_model, "models/onnx_mdl_01.onnx")

ValueError: Input 0 of layer sequential_1 is incompatible with the layer: : expected min_ndim=4, found ndim=2. Full shape received: [3, 3]

In [21]:
model.save('models/')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/assets


In [25]:
model1 = keras.models.load_model('models/')
model1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)               